In [ ]:
# Code for the coursework for the Virtual and Augmented Reality submodule in the Contemporary Computer Science IV module
# python version: 3.7.2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

# Create aliases for the headings of the IMU data for easier processing
time   = 'time'
gyro_X = ' gyroscope.X'
gyro_Y = ' gyroscope.Y'
gyro_Z = ' gyroscope.Z'
acc_X  = ' accelerometer.X'
acc_Y  = ' accelerometer.Y'
acc_Z  = ' accelerometer.Z'
mag_X  = ' magnetometer.X'
mag_Y  = ' magnetometer.Y'
mag_Z  = ' magnetometer.Z '

# X is 'depth' (forward/backward) (i.e. roll)
# Y is 'across' (left/right) (i.e. pitch)
# Z is 'up' (up/down) (i.e. yaw)


In [ ]:
# Read the IMU data from the .csv file
IMU_Data = pd.read_csv("IMUData.csv")

In [ ]:
#REMOVEME Print out the raw IMU Data
IMU_Data

In [ ]:
#REMOVEME Plot Gyroscope readings
figure(figsize=(12, 6))
plt.plot(IMU_Data[time], IMU_Data[gyro_X], label='X')
plt.plot(IMU_Data[time], IMU_Data[gyro_Y], label='Y')
plt.plot(IMU_Data[time], IMU_Data[gyro_Z], label='Z')

plt.title("Gyroscope IMU Data")
plt.xlabel("Time (s)")
plt.ylabel("Angular velocity (deg/s)")
plt.legend(loc='upper right')
plt.show()

In [ ]:
#REMOVEME Plot Accelerometer
figure(figsize=(12, 6))
plt.plot(IMU_Data[time], IMU_Data[acc_X], label='X')
plt.plot(IMU_Data[time], IMU_Data[acc_Y], label='Y')
plt.plot(IMU_Data[time], IMU_Data[acc_Z], label='Z')

plt.title("Raw Accelerometer IMU Data")
plt.xlabel("Time (s)")
plt.ylabel("Acceleration in g ($m/s^2$)")
plt.legend(loc='lower right')
plt.show()

In [ ]:
#REMOVEME Plot Magnetometer
figure(figsize=(12, 6))
plt.plot(IMU_Data[time], IMU_Data[mag_X], label='X')
plt.plot(IMU_Data[time], IMU_Data[mag_Y], label='Y')
plt.plot(IMU_Data[time], IMU_Data[mag_Z], label='Z')

plt.title("Raw Magnetometer IMU Data")
plt.xlabel("Time (s)")
plt.ylabel("Flux in Gauss (G)")
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Problem 1

# Convert rotational rate from deg/s to rad/s

# Check that data has not already been converted #REMOVEME (only for development on Jupyter)
if any(abs(IMU_Data[gyro_X]) > 10):
    IMU_Data[gyro_X] = IMU_Data[gyro_X].apply(np.radians)
    IMU_Data[gyro_Y] = IMU_Data[gyro_Y].apply(np.radians)
    IMU_Data[gyro_Z] = IMU_Data[gyro_Z].apply(np.radians)

# Normalize the accelerometer and magnetometer values

def normalize(x, y, z):
    """Normalizes the magnitude of the given [X, Y, Z] vector
    
    \in x: the x-axis value
    \in y: the y-axis value
    \in z: the z-axis value
    \out x_norm: the normalised x-axis value
    \out y_norm: the normalised y-axis value
    \out z_norm: the normalised z-axis value
    """
    magnitude = np.sqrt(x**2 + y**2 + z**2)
    
    # Make sure magnitude != 0
    if magnitude:
        return x/magnitude, y/magnitude, z/magnitude
    
    # If magnitude = 0, return the vector as-is
    return x, y, z

for index, row in IMU_Data.iterrows():
    row[acc_X], row[acc_Y], row[acc_Z] = normalize(row[acc_X], row[acc_Y], row[acc_Z])
    row[mag_X], row[mag_Y], row[mag_Z] = normalize(row[mag_X], row[mag_Y], row[mag_Z])

# Methods for converting Euler angles (radians) <---> quaternions

def euler2quaternion(eulers):
    """Converts the given euler angles into a quaternion format
    
    \in eulers: the input euler angles of format (roll, pitch, yaw)
    \out quat: the equivalent quaternion of format (w, x, y, z)
    """
    # extract the euler angles
    roll, pitch, yaw = eulers
    
    # Compute the sinusoidal values
    cos_y = np.cos(yaw/2)
    sin_y = np.sin(yaw/2)
    cos_p = np.cos(pitch/2)
    sin_p = np.sin(pitch/2)
    cos_r = np.cos(roll/2)
    sin_r = np.sin(roll/2)
    
    quat_w = cos_y * cos_p * cos_r + sin_y * sin_p * sin_r
    quat_x = cos_y * cos_p * sin_r - sin_y * sin_p * cos_r
    quat_y = sin_y * cos_p * sin_r + cos_y * sin_p * cos_r
    quat_z = sin_y * cos_p * cos_r - cos_y * sin_p * sin_r
    
    return (quat_w, quat_x, quat_y, quat_z)

def quaternion2euler(quat):
    """Converts the given quaternion into euler angle format
    
    \in quat: the input quaternion of format (w, x, y, z)
    \out euler: the equivalent euler angles of format (roll, pitch, yaw)
    """
    # extract the quaternion values
    w, x, y, z = quat
    
    # Roll
    roll  = np.arctan2(2 * (w*x + y*z), 1 - 2 * (x**2 + y**2))
    # Pitch
    asin_val = 2 * (w*y - z*x)
    if abs(asin_val) >= 1:
        # Use pi/2 as the value as the value is out of the range of arcsin
        sign = 1 if asin_val >= 0 else -1
        pitch = np.pi/2 * sign
    else:
        pitch = np.arcsin(asin_val)
    # Yaw
    yaw = np.arctan2(2 * (w*z + x*y), 1 - 2 * (y**2 + z**2))
    
    return (roll, pitch, yaw)
    

# Method for converting quaternion to its conjugate

def quaternion_conjugate(quat):
    """Convert the given quaternion to its conjugate
    
    \in quat: the input quaternion of format (w, x, y, z)
    \out out_quat: the quaternon conjugate of format (w, x, y, z)
    """
    return (quat[0], -quat[1], -quat[2], -quat[3])

# Method for calculating quaternion product of two quaternions
def quaternion_product(quat1, quat2):
    """Calculate the product of the two quaternions
    
    \in quat1: the first quaternion of format (w, x, y, z)
    \in quat2: the second quaternion of format (w, x, y, z)
    \out quat: the quaternion product of format (w, x, y, z)
    """
    # Extract the quaternion values
    w1, x1, y1, z1 = quat1
    w2, x2, y2, z2 = quat2
    
    # Construct the quaternion product
    w = w1*w2 - x1*x2 - y1*y2 - z1*z2
    x = w1*x2 + x1*w2 + y1*z2 - z1*y2
    y = w1*y2 - x1*z2 + y1*w2 + z1*x2
    z = w1*z2 + x1*y2 - y1*x2 + z1*w2
    
    return (w, x, y, z)

In [ ]:
#REMOVEME Plot Gyroscope readings
figure(figsize=(12, 6))
plt.plot(IMU_Data[time], IMU_Data[gyro_X], label='X')
plt.plot(IMU_Data[time], IMU_Data[gyro_Y], label='Y')
plt.plot(IMU_Data[time], IMU_Data[gyro_Z], label='Z')

plt.title("Gyroscope IMU Data")
plt.xlabel("Time (s)")
plt.ylabel("Angular velocity (rad/s)")
plt.legend(loc='upper right')
plt.show()

In [ ]:
#REMOVEME Plot Accelerometer
figure(figsize=(12, 6))
plt.plot(IMU_Data[time], IMU_Data[acc_X], label='X')
plt.plot(IMU_Data[time], IMU_Data[acc_Y], label='Y')
plt.plot(IMU_Data[time], IMU_Data[acc_Z], label='Z')

plt.title("Normalized Accelerometer IMU Data")
plt.xlabel("Time (s)")
plt.ylabel("Acceleration in g ($m/s^2$)")
plt.legend(loc='lower right')
plt.show()

In [ ]:
#REMOVEME Plot Magnetometer
figure(figsize=(12, 6))
plt.plot(IMU_Data[time], IMU_Data[mag_X], label='X')
plt.plot(IMU_Data[time], IMU_Data[mag_Y], label='Y')
plt.plot(IMU_Data[time], IMU_Data[mag_Z], label='Z')

plt.title("Normalized Magnetometer IMU Data")
plt.xlabel("Time (s)")
plt.ylabel("Flux in Gauss (G)")
plt.legend(loc='upper right')
plt.show()

In [ ]:
#REMOVEME
IMU_Data

In [ ]:
#REMOVEME
test_euler = (np.pi/2, 0, np.pi/2)
print("Euler: {}".format(test_euler))
test_quat = euler2quaternion(test_euler)
print("Quaternion: {}".format(test_quat))
out_euler = quaternion2euler(test_quat)
print("Euler: {}".format(out_euler))

In [ ]:
# General (non-required) utility functions

def quaternion_rotation(q, p):
    """Rotates the point p by the quaternion q"""
    if len(p) == 3:
        p = (0, p[0], p[1], p[2])
    result = quaternion_product(p, quaternion_conjugate(q))
    result = quaternion_product(q, result)
    return result

In [ ]:
# Problem 2

head_orientation = pd.DataFrame(columns=['time', 'quaternion', 'W', 'X', 'Y', 'Z']) # List of quaternions corresponding to orientation at each time stamp
head_orientation['time'] = IMU_Data['time'][:]

def simple_dead_reckoning(prev_ho_row, current_IMU_Data_row):
    """Performs simple dead reckoning to track the current head orientation by integrating
    the gyroscope readings.
    
    \in prev_ho_row: the previous head_orientation row (time and estimated quaternion)
    \in current_IMU_Data_row: the row of data in the IMU_Data frame corresponding to the current time step
    \out current_orientation: the estimated current head orientation
    """
    # Extract the data we need from the current_IMU_Data_row (row in IMU_Data frame that corresponds to the current time step)
    current_time = current_IMU_Data_row[time]
    gyroX        = current_IMU_Data_row[gyro_X]
    gyroY        = current_IMU_Data_row[gyro_Y]
    gyroZ        = current_IMU_Data_row[gyro_Z]
    
    # Extract the data we need from the prev_ho_row (previous row in the head_orientation_tilt_correction frame)
    last_orientation = prev_ho_row['quaternion']
    last_time = prev_ho_row[time]
    
    # Calculate the magnitude of the gyro readings
    l = np.sqrt(gyroX ** 2 + gyroY ** 2 + gyroZ ** 2)
    
    # Normalize the gyro readings (so they can be used as axes in axis-angle representation)
    gyroX /= l
    gyroY /= l
    gyroZ /= l
    
    # Calculate the angle of the rotation
    theta = l*(current_time - last_time)
    
    # Create the quaternion from this axis-angle representation
    change_quat = (np.cos(theta/2), gyroX*np.sin(theta/2), gyroY*np.sin(theta/2), gyroZ*np.sin(theta/2))

    # Produce the current orientation by combining (multiplying) the last orientation and the change quaternion
    current_orientation = quaternion_product(last_orientation, change_quat)
    
    return current_orientation

# Initial orientation
head_orientation.at[0, 'quaternion'] = (1, 0, 0, 0)
for index in range(len(head_orientation)):
    # Skip the first one as this has the value of the identity quaternion
    if index != 0:
        prev_row = head_orientation.iloc[index - 1]
        imu_row = IMU_Data.iloc[index]
        # Estimate the current head orientation
        head_orientation.at[index, 'quaternion'] = simple_dead_reckoning(prev_row, imu_row)
    # Set the separate W, X, Y, Z values of this row
    quat = head_orientation.iloc[index]['quaternion']
    head_orientation.at[index, 'W'] = quat[0]
    head_orientation.at[index, 'X'] = quat[1]
    head_orientation.at[index, 'Y'] = quat[2]
    head_orientation.at[index, 'Z'] = quat[3]


In [ ]:
head_orientation_euler = pd.DataFrame(columns=['time', 'X', 'Y', 'Z'])
head_orientation_euler['time'] = IMU_Data['time'][:]

for index in range(len(head_orientation)):
    quat = head_orientation.iloc[index]['quaternion']
    x, y, z = quaternion2euler(quat)
    head_orientation_euler.at[index, 'X'] = x
    head_orientation_euler.at[index, 'Y'] = y
    head_orientation_euler.at[index, 'Z'] = z

In [ ]:
#REMOVEME
figure(figsize=(12, 8))
plt.plot(head_orientation_euler[time], head_orientation_euler['X'])
plt.plot(head_orientation_euler[time], head_orientation_euler['Y'])
plt.plot(head_orientation_euler[time], head_orientation_euler['Z'])

plt.axhline(0, color='black', alpha=0.3)
plt.title("Dead Reckoning Filter for Head Orientation Tracking")
plt.xlabel("Time (s)")
plt.ylabel("Angle component (rad)")
plt.legend(loc='upper right')
plt.show()

#REMOVEME
figure(figsize=(12, 8))
plt.plot(head_orientation[time], head_orientation['X'])
plt.plot(head_orientation[time], head_orientation['Y'])
plt.plot(head_orientation[time], head_orientation['Z'])

plt.axhline(0, color='black', alpha=0.3)
plt.title("Quaternion: Dead Reckoning Filter for Head Orientation Tracking")
plt.xlabel("Time (s)")
plt.ylabel("Angle component (rad)")
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Problem 3

head_orientation_tilt_correction = pd.DataFrame(columns=['time', 'quaternion', 'W', 'X', 'Y', 'Z']) # List of quaternions corresponding to orientation at each time stamp
head_orientation_tilt_correction['time'] = IMU_Data['time'][:]

def tilt_correction(prev_ho_row, current_IMU_Data_row):
    """Estimates the head orientation with additionally applying gravity-based tilt correction
    
    \in prev_ho_row: the row of data (time and quaternion) corresponding to the last time step
    \in current_IMU_Data_row: the row of data in the IMU_Data frame that corresponds to the current time step
    \out tilt_corrected_orientation: the estimated head orientation, with gravity-based tilt correction
    """
    # Extract the data we need from current_IMU_Data_row
    accX = current_IMU_Data_row[acc_X]
    accY = current_IMU_Data_row[acc_Y]
    accZ = current_IMU_Data_row[acc_Z]
        
    # Get the result from simple dead reckoning
    simple_orientation = simple_dead_reckoning(prev_ho_row, current_IMU_Data_row)
    
    # Perform tilt correction
    
    # Transform acceleration measurements into the global frame
    acc_quat = (0, accX, accY, accZ) # convert the acc vector into a quaternion as per LaValle paper
    a_global = quaternion_product(acc_quat, quaternion_conjugate(simple_orientation))
    a_global = quaternion_product(simple_orientation, a_global)
    
    # Calculate the tilt axis - this is orthogonal to the XY plane (as Z is 'up'), so we take it as (-a_y, a_x, 0)
    tilt_axis = (-a_global[2], a_global[1], 0)
    
    # Find the angle phi between the up vector (0, 0, 1) and the accelerometer vector (a_x, a_y, a_z)
    # phi = arccos(dot_product(a, up)/(magnitude(a)*magnitude(up))). However, magnitude of both a and up are 1, as a is normalized
    # and up is (0, 0, 1). Also, since up is (0, 0, 1) the dot product of a and up is simply a_z. Thus:
    phi = np.arccos(a_global[3])
    
    # Use the complementary filter to fuse the two signals
    alpha = 0.03
    # Construct the tilt quaternion
    theta = -alpha * phi
    tilt_quat = (np.cos(theta/2), tilt_axis[0]*np.sin(theta/2), tilt_axis[1]*np.sin(theta/2), tilt_axis[2]*np.sin(theta/2))
    
    tilt_corrected_orientation = quaternion_product(tilt_quat, simple_orientation)
    
    return tilt_corrected_orientation

# Initial orientation
head_orientation_tilt_correction.at[0, 'quaternion'] = (1, 0, 0, 0)
for index in range(len(head_orientation_tilt_correction)):
    # Skip the first one as this has the value of the identity quaternion
    if index != 0:
        prev_row = head_orientation_tilt_correction.iloc[index - 1]
        imu_row = IMU_Data.iloc[index]
        # Estimate the current head orientation, with tilt correction
        head_orientation_tilt_correction.at[index, 'quaternion'] = tilt_correction(prev_row, imu_row)
    # Set the separate W, X, Y, Z values of this row
    quat = head_orientation_tilt_correction.iloc[index]['quaternion']
    head_orientation_tilt_correction.at[index, 'W'] = quat[0]
    head_orientation_tilt_correction.at[index, 'X'] = quat[1]
    head_orientation_tilt_correction.at[index, 'Y'] = quat[2]
    head_orientation_tilt_correction.at[index, 'Z'] = quat[3]


In [ ]:
head_orientation_tilt_correction_euler = pd.DataFrame(columns=['time', 'X', 'Y', 'Z'])
head_orientation_tilt_correction_euler['time'] = IMU_Data['time'][:]

for index in range(len(head_orientation_tilt_correction)):
    quat = head_orientation_tilt_correction.iloc[index]['quaternion']
    x, y, z = quaternion2euler(quat)
    head_orientation_tilt_correction_euler.at[index, 'X'] = x
    head_orientation_tilt_correction_euler.at[index, 'Y'] = y
    head_orientation_tilt_correction_euler.at[index, 'Z'] = z

In [ ]:
#REMOVEME alpha = 0.03
figure(figsize=(12, 8))
plt.plot(head_orientation_tilt_correction_euler[time], head_orientation_tilt_correction_euler['X'])
plt.plot(head_orientation_tilt_correction_euler[time], head_orientation_tilt_correction_euler['Y'])
plt.plot(head_orientation_tilt_correction_euler[time], head_orientation_tilt_correction_euler['Z'])

plt.axhline(0, color='black', alpha=0.3)
plt.title("Tilt Correction for Head Orientation Tracking, alpha = 0.03")
plt.xlabel("Time (s)")
plt.ylabel("Angle component (rad)")
plt.legend(loc='upper right')
plt.show()

figure(figsize=(12, 8))
plt.plot(head_orientation_tilt_correction[time], head_orientation_tilt_correction['X'])
plt.plot(head_orientation_tilt_correction[time], head_orientation_tilt_correction['Y'])
plt.plot(head_orientation_tilt_correction[time], head_orientation_tilt_correction['Z'])

plt.axhline(0, color='black', alpha=0.3)
plt.title("Quaternion: Tilt Correction for Head Orientation Tracking, alpha = 0.03")
plt.xlabel("Time (s)")
plt.ylabel("Angle component (rad)")
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Problem 4

head_orientation_yaw_correction = pd.DataFrame(columns=['time', 'quaternion', 'W', 'X', 'Y', 'Z']) # List of quaternions corresponding to orientation at each time stamp
head_orientation_yaw_correction['time'] = IMU_Data['time'][:]

def yaw_correction(prev_ho_row, current_IMU_Data_row):
    """Estimates the head orientation with additionally applying gravity-based tilt correction
    
    \in prev_ho_row: the row of data (time and quaternion) corresponding to the last time step
    \in current_IMU_Data_row: the row of data in the IMU_Data frame that corresponds to the current time step
    \out tilt_corrected_orientation: the estimated head orientation, with gravity-based tilt correction
    """
    # Extract the data we need from current_IMU_Data_row
    magX = current_IMU_Data_row[mag_X]
    magY = current_IMU_Data_row[mag_Y]
    magZ = current_IMU_Data_row[mag_Z]
    
    # Get the result from tilt correction
    tilt_corrected_orientation = tilt_correction(prev_ho_row, current_IMU_Data_row)
    
    # Perform yaw correction
    
    # Get reference readings
    mag_ref = (0, IMU_Data.iloc[0][mag_X], IMU_Data.iloc[0][mag_Y], IMU_Data.iloc[0][mag_Z])
    quat_ref = head_orientation_yaw_correction.iloc[0]['quaternion']
    
    mag_current = (0, magX, magY, magZ)
    
    # Bring the current readings into the estimated global frame
    q = tilt_corrected_orientation
    q_inv = quaternion_conjugate(tilt_corrected_orientation)
    mag_current = quaternion_product(q, quaternion_product(mag_current, q_inv))
    
    # Find the angular difference between mag_current and mag_ref, when projected into the XY plane
    theta_current = np.arctan2(mag_current[1], mag_current[2])
    theta_ref = np.arctan2(mag_ref[1], mag_ref[2])
    
    # Apply the same complementary filter to fix the yaw drift
    alpha = 0.00021
    # Construct the correction quaternion
    theta = -alpha * (theta_current - theta_ref)
    correction_quat = (np.cos(theta/2), 0, 0, np.sin(theta/2))
    
    yaw_corrected_orientation = quaternion_product(correction_quat, tilt_corrected_orientation) #if (theta_current - theta_ref) > 
    
    return yaw_corrected_orientation

# Initial orientation
head_orientation_yaw_correction.at[0, 'quaternion'] = (1, 0, 0, 0)
for index in range(len(head_orientation_yaw_correction)):
    # Skip the first one as this has the value of the identity quaternion
    if index != 0:
        prev_row = head_orientation_yaw_correction.iloc[index - 1]
        imu_row = IMU_Data.iloc[index]
        # Estimate the current head orientation, with tilt correction
        head_orientation_yaw_correction.at[index, 'quaternion'] = yaw_correction(prev_row, imu_row)
    # Set the separate W, X, Y, Z values of this row
    quat = head_orientation_yaw_correction.iloc[index]['quaternion']
    head_orientation_yaw_correction.at[index, 'W'] = quat[0]
    head_orientation_yaw_correction.at[index, 'X'] = quat[1]
    head_orientation_yaw_correction.at[index, 'Y'] = quat[2]
    head_orientation_yaw_correction.at[index, 'Z'] = quat[3]


In [ ]:
head_orientation_yaw_correction_euler = pd.DataFrame(columns=['time', 'X', 'Y', 'Z'])
head_orientation_yaw_correction_euler['time'] = IMU_Data['time'][:]

for index in range(len(head_orientation_yaw_correction)):
    quat = head_orientation_yaw_correction.iloc[index]['quaternion']
    x, y, z = quaternion2euler(quat)
    head_orientation_yaw_correction_euler.at[index, 'X'] = x
    head_orientation_yaw_correction_euler.at[index, 'Y'] = y
    head_orientation_yaw_correction_euler.at[index, 'Z'] = z

In [ ]:
#REMOVEME
figure(figsize=(12, 8))
plt.plot(head_orientation_yaw_correction_euler[time], head_orientation_yaw_correction_euler['X'])
plt.plot(head_orientation_yaw_correction_euler[time], head_orientation_yaw_correction_euler['Y'])
plt.plot(head_orientation_yaw_correction_euler[time], head_orientation_yaw_correction_euler['Z'])

plt.axhline(0, color='black', alpha=0.3)
plt.title("Yaw Correction for Head Orientation Tracking, alpha = 0.0002")
plt.xlabel("Time (s)")
plt.ylabel("Angle component (rad)")
plt.legend(loc='upper right')
plt.show()

figure(figsize=(12, 8))
plt.plot(head_orientation_yaw_correction[time], head_orientation_yaw_correction['X'])
plt.plot(head_orientation_yaw_correction[time], head_orientation_yaw_correction['Y'])
plt.plot(head_orientation_yaw_correction[time], head_orientation_yaw_correction['Z'])

plt.axhline(0, color='black', alpha=0.3)
plt.title("Quaternion: Yaw Correction for Head Orientation Tracking, alpha = 0.0002")
plt.xlabel("Time (s)")
plt.ylabel("Angle component (rad)")
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Problem 5 - 2D plots

# Tri-axial angular rate in deg/s
figure(figsize=(12, 4))
plt.plot(IMU_Data[time], np.degrees(IMU_Data[gyro_X]), label='X')
plt.plot(IMU_Data[time], np.degrees(IMU_Data[gyro_Y]), label='Y')
plt.plot(IMU_Data[time], np.degrees(IMU_Data[gyro_Z]), label='Z')

plt.title("Tri-axial angular rate (Gyroscope)")
plt.xlabel("Time (s)")
plt.ylabel("Angular rate ($deg/s$)")
plt.legend(loc='upper right')
plt.show()

# Tri-axial acceleration in g (m/s^2)
figure(figsize=(12, 4))
plt.plot(IMU_Data[time], IMU_Data[acc_X], label='X')
plt.plot(IMU_Data[time], IMU_Data[acc_Y], label='Y')
plt.plot(IMU_Data[time], IMU_Data[acc_Z], label='Z')

plt.title("Tri-axial acceleration (Accelerometer)")
plt.xlabel("Time (s)")
plt.ylabel("Acceleration in $g$ ($m/s^2$)")
plt.legend(loc='upper right')
plt.show()

# Tri-axial magnetic flux in Gauss (G)
figure(figsize=(12, 4))
plt.plot(IMU_Data[time], IMU_Data[mag_X], label='X')
plt.plot(IMU_Data[time], IMU_Data[mag_Y], label='Y')
plt.plot(IMU_Data[time], IMU_Data[mag_Z], label='Z')

plt.title("Tri-axial magnetic flux (Magnetometer)")
plt.xlabel("Time (s)")
plt.ylabel("Magnetic flux in Gauss ($G$)")
plt.legend(loc='upper right')
plt.show()

# Tri-axial Euler angles in degrees for gyroscope integration, gyroscope + accelerometer, gyroscope + accelerometer + magnetometer
# %matplotlib qt # Make the output pop-out and be interactive

figure(figsize=(12, 12))
plt.tight_layout()
plt.subplot(3, 1, 1)
# Gyroscope
plt.plot(head_orientation_euler[time], np.degrees(np.float32(head_orientation_euler['X'])), label='X')
plt.plot(head_orientation_euler[time], np.degrees(np.float32(head_orientation_euler['Y'])), label='Y')
plt.plot(head_orientation_euler[time], np.degrees(np.float32(head_orientation_euler['Z'])), label='Z')
plt.axhline(0, color='black', alpha=0.3)
plt.title("Gyroscope Integration")
plt.xlabel("Time (s)")
plt.ylabel("Euler angle (deg)")
plt.legend(loc='upper right')

plt.subplot(3, 1, 2)
# Gyroscope + Accelerometer
plt.plot(head_orientation_tilt_correction_euler[time], np.degrees(np.float32(head_orientation_tilt_correction_euler['X'])), label='X')
plt.plot(head_orientation_tilt_correction_euler[time], np.degrees(np.float32(head_orientation_tilt_correction_euler['Y'])), label='Y')
plt.plot(head_orientation_tilt_correction_euler[time], np.degrees(np.float32(head_orientation_tilt_correction_euler['Z'])), label='Z')
plt.axhline(0, color='black', alpha=0.3)
plt.title("Gyroscope + Accelerometer")
plt.xlabel("Time (s)")
plt.ylabel("Euler angle (deg)")
plt.legend(loc='upper right')

plt.subplot(3, 1, 3)
# Gyroscope + Accelerometer + Magnetometer
plt.plot(head_orientation_yaw_correction_euler[time], np.degrees(np.float32(head_orientation_yaw_correction_euler['X'])), label='X')
plt.plot(head_orientation_yaw_correction_euler[time], np.degrees(np.float32(head_orientation_yaw_correction_euler['Y'])), label='Y')
plt.plot(head_orientation_yaw_correction_euler[time], np.degrees(np.float32(head_orientation_yaw_correction_euler['Z'])), label='Z')
plt.axhline(0, color='black', alpha=0.3)
plt.title("Gyroscope + Accelerometer + Magnetometer")
plt.xlabel("Time (s)")
plt.ylabel("Euler angle (deg)")
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()


In [ ]:
# Problem 5 - 3D plots
# Make plots pop-out and be interactive
%matplotlib qt
# 3D plots of the 3 main axis of the IMU for gyroscope integration, gyroscope + acceleration, gyroscope + acceleration + magnetometer

import mpl_toolkits.mplot3d.axes3d as p3
from matplotlib import animation

N = 6959 # Number of frames
interval = 4 # Time between frames in ms - 1000/256 as time steps are at 256Hz
resolution = 5 # How often frames are generated (i.e. resolution=4 means every 4th data point is plotted)
# Set the total number of frames as adjusted by resolution
N //= resolution
scale = 2

axisX, axisY, axisZ = [0, 0, 0], [0, 0, 0], [0, 0, 0]

def animate_axes(num, ax1, ax2, ax3):
    """Generate the new quivers"""
    global axisX, axisY, axisZ
    # Check that this isn't the first frame
    if axisX[0] != 0:
        for i in axisX, axisY, axisZ:
            for j in i:
                j.remove()
        
    num *= resolution
    
    x_proj_vec = (0, 1, 0, 0)
    y_proj_vec = (0, 0, 1, 0)
    z_proj_vec = (0, 0, 0, 1)
    # Apply the current quaternion (indexed by num) to these vectors
    quaternion = head_orientation.iloc[num]['quaternion']
    tilt_corrected_quaternion = head_orientation_tilt_correction.iloc[num]['quaternion']
    yaw_corrected_quaternion = head_orientation_yaw_correction.iloc[num]['quaternion']
    
    axes = [ax1, ax2, ax3]
    quats = [quaternion, tilt_corrected_quaternion, yaw_corrected_quaternion]
    
    for i in range(3):
        ax = axes[i]
        quat = quats[i]
        
#         ax.cla()
#         Setting the axes properties

        x_vec = quaternion_rotation(quat, x_proj_vec)
        y_vec = quaternion_rotation(quat, y_proj_vec)
        z_vec = quaternion_rotation(quat, z_proj_vec)

        axisX[i] = ax.quiver(0, 0, 0, *x_vec[1:], color='Blue', label='X')
        axisY[i] = ax.quiver(0, 0, 0, *y_vec[1:], color='Orange', label='Y')
        axisZ[i] = ax.quiver(0, 0, 0, *z_vec[1:], color='Green', label='Z')
        
        ax.legend(loc='center left')

fig = plt.figure(figsize=(24, 8))
# Gyroscope integration
ax1 = fig.add_subplot(131, projection='3d')
ax1.set_title("Gyroscope integration")
ax1.set_xlim3d([-1.0*scale, 1.0*scale]); ax1.set_xlabel('X')
ax1.set_ylim3d([-1.0*scale, 1.0*scale]); ax1.set_ylabel('Y')
ax1.set_zlim3d([-1.0*scale, 1.0*scale]); ax1.set_zlabel('Z')

# Gyroscope + Accelerometer
ax2 = fig.add_subplot(132, projection='3d')
ax2.set_title("Gyroscope + Accelerometer")
ax2.set_xlim3d([-1.0*scale, 1.0*scale]); ax2.set_xlabel('X')
ax2.set_ylim3d([-1.0*scale, 1.0*scale]); ax2.set_ylabel('Y')
ax2.set_zlim3d([-1.0*scale, 1.0*scale]); ax2.set_zlabel('Z')

# Gyroscope + Accelerometer + Magnetometer
ax3 = fig.add_subplot(133, projection='3d')
ax3.set_title("Gyroscope + Accelerometer + Magnetometer")
ax3.set_xlim3d([-1.0*scale, 1.0*scale]); ax3.set_xlabel('X')
ax3.set_ylim3d([-1.0*scale, 1.0*scale]); ax3.set_ylabel('Y')
ax3.set_zlim3d([-1.0*scale, 1.0*scale]); ax3.set_zlabel('Z')

ani = animation.FuncAnimation(fig, animate_axes, N, fargs=(ax1, ax2, ax3), interval=interval, blit=False)

plt.tight_layout()
plt.show()

In [ ]:
# Problem 6

# Gyroscope Integration
head_position = pd.DataFrame(columns=['time', 'posGlobal', 'posBody', 'velocity', 'quaternion', 'W', 'X', 'Y', 'Z'])
# Copy data from the simple head orientation estimate
for col in ['time', 'quaternion', 'W', 'X', 'Y', 'Z']:
    head_position[col] = head_orientation[col][:]

# Gyroscope + Accelerometer
head_position_tilt = pd.DataFrame(columns=['time', 'posGlobal', 'posBody', 'velocity', 'quaternion', 'W', 'X', 'Y', 'Z'])
# Copy data from the tilt corrected head orientation estimate
for col in ['time', 'quaternion', 'W', 'X', 'Y', 'Z']:
    head_position_tilt[col] = head_orientation_tilt_correction[col][:]

# Gyroscope + Accelerometer + Magnetometer
head_position_yaw = pd.DataFrame(columns=['time', 'posGlobal', 'posBody', 'velocity', 'quaternion', 'W', 'X', 'Y', 'Z'])
# Copy data from the yaw corrected head orientation estimate
for col in ['time', 'quaternion', 'W', 'X', 'Y', 'Z']:
    head_position_yaw[col] = head_orientation_yaw_correction[col][:]

l_n = 0.015 # Distance from height of pupils to the base of the neck (0.15m, i.e. 0.015 as 1 = 10m (g))
l_n = 0.25

def positional_tracking(prev_pos_row, current_IMU_Data_row):
    # Extract the raw accelerometer readings from the IMU_Data row
    accX = current_IMU_Data_row[acc_X]
    accY = current_IMU_Data_row[acc_Y]
    accZ = current_IMU_Data_row[acc_Z]
    
    # Extract data from prev_pos_row
    head_orientation = prev_pos_row['quaternion']
    prev_vel = prev_pos_row['velocity']
    prev_body_pos = prev_pos_row['posBody']
    
    t = current_IMU_Data_row['time'] - prev_pos_row['time']
    
    # Convert accelerometer readings to global frame
    acc_quat = (0, accX, accY, accZ)
    
    acc_global = quaternion_rotation(head_orientation, acc_quat)

    # Subtract 1 (gravity) from Z acceleration to get (estimated) linear acceleration
    acc_global = (acc_global[1], acc_global[2], acc_global[3] - 1)

    # Double integrate global accelerometer readings to produce body position
    # Using r = r_0 + v_0*t + 0.5*a(t^2), where r = position
    body_posX = prev_body_pos[0] + prev_vel[0]*t + 0.5*acc_global[0]*t*t
    body_posY = prev_body_pos[1] + prev_vel[1]*t + 0.5*acc_global[1]*t*t
    body_posZ = prev_body_pos[2] + prev_vel[2]*t + 0.5*acc_global[2]*t*t
    
    # Find the updated velocity
    new_velX = prev_vel[0] + acc_global[0]*t
    new_velY = prev_vel[1] + acc_global[1]*t
    new_velZ = prev_vel[2] + acc_global[2]*t
    
    body_pos = (body_posX, body_posY, body_posZ)

    # Apply kinematic head model to produce head position
    head_pos = (0, 0, 0, l_n)
    head_pos = quaternion_rotation(head_orientation, head_pos)[1:]
    
    # Compose body and head position to produce the final position
    global_pos = np.add(head_pos, body_pos)
    
    # Return all required values
    return global_pos, body_pos, (new_velX, new_velY, new_velZ)

# Initial values - position = (0, 0, ln), body position = (0, 0, 0), velocity = (0, 0, 0)
for table in [head_position, head_position_tilt, head_position_yaw]:
    table.at[0, 'posGlobal'] = (0, 0, l_n)
    table.at[0, 'posBody'] = (0, 0, 0)
    table.at[0, 'velocity'] = (0, 0, 0)

for index in range(len(head_position)):
    # Skip the first one as this has the value of the identity quaternion
    if index != 0:
        prev_row = head_position.iloc[index - 1]
        imu_row = IMU_Data.iloc[index]
        # Estimate the current head position, with tilt correction
        pos_global, posBody, velocity = positional_tracking(prev_row, imu_row)
        head_position.at[index, 'posGlobal'] = pos_global
        head_position.at[index, 'posBody'] = posBody
        head_position.at[index, 'velocity'] = velocity

# Gyroscope + Accelerometer
for index in range(len(head_position_tilt)):
    # Skip the first one as this has the value of the identity quaternion
    if index != 0:
        prev_row = head_position_tilt.iloc[index - 1]
        imu_row = IMU_Data.iloc[index]
        # Estimate the current head position, with tilt correction
        pos_global, posBody, velocity = positional_tracking(prev_row, imu_row)
        head_position_tilt.at[index, 'posGlobal'] = pos_global
        head_position_tilt.at[index, 'posBody'] = posBody
        head_position_tilt.at[index, 'velocity'] = velocity

# Gyroscope + Accelerometer + Magnetometer
for index in range(len(head_position_yaw)):
    # Skip the first one as this has the value of the identity quaternion
    if index != 0:
        prev_row = head_position_yaw.iloc[index - 1]
        imu_row = IMU_Data.iloc[index]
        # Estimate the current head position, with tilt correction
        pos_global, posBody, velocity = positional_tracking(prev_row, imu_row)
        head_position_yaw.at[index, 'posGlobal'] = pos_global
        head_position_yaw.at[index, 'posBody'] = posBody
        head_position_yaw.at[index, 'velocity'] = velocity

In [ ]:
# Problem 6 - 3D plots

# 3D plots of the 3 main axis rotations + translations of the IMU for gyroscope integration, gyroscope + acceleration, gyroscope + acceleration + magnetometer

import mpl_toolkits.mplot3d.axes3d as p3
from matplotlib import animation

N = 6959 # Number of frames
interval = 4 # Time between frames in ms - 1000/256 as time steps are at 256Hz
resolution = 5 # How often frames are generated (i.e. resolution=4 means every 4th data point is plotted)
# Set the total number of frames as adjusted by resolution
N //= resolution
scale = 2

axisX, axisY, axisZ = [0, 0, 0], [0, 0, 0], [0, 0, 0]

def animate_axes(num, ax1, ax2, ax3):
    """Generate the new quivers"""
    global axisX, axisY, axisZ
    # Check that this isn't the first frame
    if axisX[0] != 0:
        for i in axisX, axisY, axisZ:
            for j in i:
                j.remove()
        
    num *= resolution
    
    x_proj_vec = (0, 1, 0, 0)
    y_proj_vec = (0, 0, 1, 0)
    z_proj_vec = (0, 0, 0, 1)
    # Apply the current quaternion (indexed by num) to these vectors
    quaternion = head_orientation.iloc[num]['quaternion']
    tilt_corrected_quaternion = head_orientation_tilt_correction.iloc[num]['quaternion']
    yaw_corrected_quaternion = head_orientation_yaw_correction.iloc[num]['quaternion']
    pos_simple = head_position.iloc[num]['posGlobal']
    pos_tilt   = head_position_tilt.iloc[num]['posGlobal']
    pos_yaw    = head_position_yaw.iloc[num]['posGlobal']
    
    axes = [ax1, ax2, ax3]
    quats = [quaternion, tilt_corrected_quaternion, yaw_corrected_quaternion]
    poss = [pos_simple, pos_tilt, pos_yaw]
    
    for i in range(3):
        ax = axes[i]
        quat = quats[i]
        pos = poss[i]
        
        x_vec = quaternion_rotation(quat, x_proj_vec)
        y_vec = quaternion_rotation(quat, y_proj_vec)
        z_vec = quaternion_rotation(quat, z_proj_vec)

        # Add the position tracking
        axisX[i] = ax.quiver(*pos, *x_vec[1:], color='Blue', label='X')
        axisY[i] = ax.quiver(*pos, *y_vec[1:], color='Orange', label='Y')
        axisZ[i] = ax.quiver(*pos, *z_vec[1:], color='Green', label='Z')
        
        ax.legend(loc='center left')

fig = plt.figure(figsize=(24, 8))
# Gyroscope integration
ax1 = fig.add_subplot(131, projection='3d')
ax1.set_title("Gyroscope integration")
ax1.set_xlim3d([-1.0*scale, 1.0*scale]); ax1.set_xlabel('X')
ax1.set_ylim3d([-1.0*scale, 1.0*scale]); ax1.set_ylabel('Y')
ax1.set_zlim3d([-1.0*scale, 1.0*scale]); ax1.set_zlabel('Z')

# Gyroscope + Accelerometer
ax2 = fig.add_subplot(132, projection='3d')
ax2.set_title("Gyroscope + Accelerometer")
ax2.set_xlim3d([-1.0*scale, 1.0*scale]); ax2.set_xlabel('X')
ax2.set_ylim3d([-1.0*scale, 1.0*scale]); ax2.set_ylabel('Y')
ax2.set_zlim3d([-1.0*scale, 1.0*scale]); ax2.set_zlabel('Z')

# Gyroscope + Accelerometer + Magnetometer
ax3 = fig.add_subplot(133, projection='3d')
ax3.set_title("Gyroscope + Accelerometer + Magnetometer")
ax3.set_xlim3d([-1.0*scale, 1.0*scale]); ax3.set_xlabel('X')
ax3.set_ylim3d([-1.0*scale, 1.0*scale]); ax3.set_ylabel('Y')
ax3.set_zlim3d([-1.0*scale, 1.0*scale]); ax3.set_zlabel('Z')

ani = animation.FuncAnimation(fig, animate_axes, N, fargs=(ax1, ax2, ax3), interval=interval, blit=False)

plt.tight_layout()
plt.show()

In [ ]:
# #REMOVEME

# %matplotlib inline

# x_yaw  = [i[0] for i in head_position_yaw['posGlobal']]
# y_yaw  = [i[1] for i in head_position_yaw['posGlobal']]
# z_yaw  = [i[2] for i in head_position_yaw['posGlobal']]
# x_tilt = [i[0] for i in head_position_tilt['posGlobal']]
# y_tilt = [i[1] for i in head_position_tilt['posGlobal']]
# z_tilt = [i[2] for i in head_position_tilt['posGlobal']]
# x_gyro = [i[0] for i in head_position['posGlobal']]
# y_gyro = [i[1] for i in head_position['posGlobal']]
# z_gyro = [i[2] for i in head_position['posGlobal']]


# figure(figsize=(12, 8))
# plt.plot(head_orientation_yaw_correction[time], x_yaw, label='X yaw')
# plt.plot(head_orientation_yaw_correction[time], y_yaw, label='Y yaw')
# plt.plot(head_orientation_yaw_correction[time], z_yaw, label='Z yaw')
# plt.plot(head_orientation_yaw_correction[time], x_tilt, label='X tilt')
# plt.plot(head_orientation_yaw_correction[time], y_tilt, label='Y tilt')
# plt.plot(head_orientation_yaw_correction[time], z_tilt, label='Z tilt')
# plt.plot(head_orientation_yaw_correction[time], x_gyro, label='X gyro')
# plt.plot(head_orientation_yaw_correction[time], y_gyro, label='Y gyro')
# plt.plot(head_orientation_yaw_correction[time], z_gyro, label='Z gyro')

# plt.axhline(0, color='black', alpha=0.3)
# plt.title("Position Tracking Global Positions")
# plt.xlabel("Time (s)")
# plt.ylabel("Position (m)")
# plt.legend(loc='upper right')
# plt.show()